In [1]:
from src.util import load_data, createSamples, divideTrainTest, create_multi_ahead_samples
from src.ts_loader import Time_Series_Data
from src.NN_forecasting import single_model_forecasting
from src.NN_train import train, predict, predict_iteration
from src import eval


Using TensorFlow backend.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader


import time
import numpy as np
import pandas as pd

torch.manual_seed(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
from Transformer import *

## Load Dataset

In [42]:
crop_df = pd.read_csv('./dataset/brinjal_price.csv', index_col=0, parse_dates=True, low_memory=False)


def data_preprocess(data, lag, h_train, h_test):
    trainData, testData = divideTrainTest(data)
    rnn_format = True

    trainX, trainY = create_multi_ahead_samples(trainData, lag, h_train, RNN=rnn_format)
    testX, testY = create_multi_ahead_samples(testData, lag, h_test, RNN=rnn_format)

    trainY = np.squeeze(trainY).reshape(-1, h_train)
    testY = np.squeeze(testY).reshape(-1, h_test)
    return trainX, trainY, testX, testY


train_x_arr, train_y_arr, test_x_arr, test_y_arr = data_preprocess(crop_df[crop_df.columns[0]], lag=365, h_train=1, h_test=1)

In [43]:
train_x_arr.shape

(2647, 365)

## Engineer the train/test array

In [44]:
for ix, col in enumerate(crop_df.columns):
    # ts, data = load_data('./dataset/brinjal_price.csv', columnName=col)
    if ix == 0:
        continue
    print(f"col: {col}")
    data = crop_df[col].values.reshape(-1, 1).astype("float32")
    train_x, train_y, test_x, test_y = data_preprocess(crop_df[col], lag=365, h_train=1, h_test=1)
    train_x_arr = np.concatenate((train_x_arr, train_x), axis=0)
    train_y_arr = np.concatenate((train_y_arr, train_y), axis=0)
    test_x_arr = np.concatenate((test_x_arr, test_x), axis=0)
    test_y_arr = np.concatenate((test_y_arr, test_y), axis=0)

#     print(train_x)
#     print(train_x.shape)
    

col: Cachar_Assa
col: Howly_Assa
col: Lanka_Assa
col: P.O. Uparhali Guwahati_Assa
col: Ahmedabad_Guja
col: Anand(Veg,Yard,Anand)_Guja
col: Bardoli_Guja
col: Bharuch_Guja
col: Bilimora_Guja
col: Botad_Guja
col: Dahod(Veg. Market)_Guja
col: Deesa(Deesa Veg Yard)_Guja
col: Dhari_Guja
col: Dholka_Guja
col: Kapadvanj_Guja
col: Mansa(Manas Veg Yard)_Guja
col: Morbi_Guja
col: Padra_Guja
col: Patan(Veg,Yard Patan)_Guja
col: S.Mandvi_Guja
col: Surat_Guja
col: Vadhvan_Guja
col: Vadodara(Sayajipura)_Guja
col: Vyra_Guja
col: Ambala Cantt._Hary
col: Ambala City_Hary
col: Asandh_Hary
col: Barwala_Hary
col: Bhuna_Hary
col: Ch. Dadri_Hary
col: Faridabad_Hary
col: Fatehabad_Hary
col: Gohana_Hary
col: Jagadhri_Hary
col: kalanwali_Hary
col: Khanina_Hary
col: Naraingarh_Hary
col: Palwal_Hary
col: Radaur_Hary
col: Rania_Hary
col: Ratia_Hary
col: Shahzadpur_Hary
col: Sirsa_Hary
col: Sonepat_Hary
col: Uklana_Hary
col: Yamuna Nagar_Hary
col: Bhuntar_Hima
col: Bilaspur_Hima
col: Dhanotu (Mandi)_Hima
col: Hamir

col: Rampurmaniharan_Utta
col: Sikanderabad_Utta
col: Sikarpur_Utta
col: Sultanpur_Utta
col: Thanabhawan_Utta
col: Tilhar_Utta
col: Rudrapur_Uttr
col: Alipurduar_West
col: Baxirhat_West
col: Diamond Harbour_West
col: Falakata(Dhupguri)_West
col: Guskara_West
col: Habra_West
col: Indus_West
col: Khatra_West
col: Midnapore Sadar_West
col: Samsi_West
col: Kovvur_Andh
col: Mustafabad_Hary
col: Narnaul_Hary
col: Kangra(Jaisinghpur)_Hima
col: Reasi_Jamm
col: Neeleswaram_Kera
col: Perambra_Kera
col: Quilandy_Kera
col: Ranniangadi_Kera
col: Haatpipliya_Madh
col: Barikpur_Odis
col: Junagarh_Odis
col: Kasinagar_Odis
col: Khunthabandha_Odis
col: Padampur_Odis
col: Saraskana_Odis
col: Sohela_Odis
col: Dudhansadhan_Punj
col: Mamdot_Punj
col: Moonak_Punj
col: Rampuraphul(Nabha Mandi)_Punj
col: Jodhpur(F&V)(Paota)_Raja
col: Ibrahimputnam_Tela
col: Kalwakurthy_Tela
col: Garjee_Trip
col: Jumpuijala_Trip
col: Melaghar_Trip
col: Mohanpur_Trip
col: Santir Bazar_Trip
col: Fatehpur Sikri_Utta
col: Khatauli_

In [67]:
test_x_arr.shape

(426213, 365)

In [49]:
# save array
np.save('train_x.npy', train_x_arr)
np.save('train_y.npy', train_y_arr)
np.save('test_x.npy', test_x_arr)
np.save('test_y.npy', test_y_arr)


In [69]:
train_x = np.reshape(train_x_arr, (1765549, 365, 1))
test_x = np.reshape(test_x_arr, (426213, 365, 1))

In [70]:
# dataset loader
batch_size = 128
train_dataset = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y_arr))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, sampler=None,
                                             batch_sampler=None, num_workers=1)


In [71]:
class LSTM(nn.Module):
    def __init__(self, inputDim, hiddenNum, outputDim, layerNum, drop_out=0.):
        
        super(LSTM, self).__init__()
        # hidden cell numbers
        self.hiddenNum = hiddenNum
        # input dimension
        self.inputDim = inputDim
        # output dimension
        self.outputDim = outputDim
        # layer number
        self.layerNum = layerNum
        # LSTM cell
        self.cell = nn.LSTM(input_size=self.inputDim, hidden_size=self.hiddenNum,
                               num_layers=self.layerNum, dropout=0.0,
                               batch_first=True, )
        
        print(self.cell)
        self.fc = nn.Linear(self.hiddenNum, self.outputDim)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = Variable(torch.zeros(self.layerNum * 1, batch_size, self.hiddenNum))
        c0 = Variable(torch.zeros(self.layerNum * 1, batch_size, self.hiddenNum))
#         if self.cuda:
#             h0.cuda()
#             c0.cuda()
        # lstm cell
        output, hn = self.cell(x, (h0, c0))
        hn = hn[0].view(batch_size, self.hiddenNum)
        fc_output = self.fc(hn)
        return fc_output 

In [ ]:
def train(train_loader, model, epochs, lr):
    # loss function
    criterion = nn.MSELoss()
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # track loss list
    loss_list = []
    total_steps = len(train_loader)
    
    # start training
    model.train()
    
    for epoch in epochs:
        for ix, (x, y) in enumerate(train_loader):
            # track starting time
            start_time = time.time()
            
            # clear the accumulated gradient before each instance
            model.zero_grad()
            
            # prepare the data and label
            x, y = x.to(device), y.to(device)
            
            # run the forward pass
            outputs = model(x)
            
            # Compute the loss, gradients, and update the parameters 
            loss = criterion(outputs, y.float())
            
            # back propogation
            loss.backward()
            # update parameters
            optimizer.step()
            # clear gradient
            optimizer.zero_grad()
            
            # append loss to the loss list
            loss_list.append(loss)
            
            # print in every 50 episodes
            if (ix+1) % 50 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{total_steps}], '
                   f'Time [{time.time() - start_time} sec], Avg loss: {sum(loss_list[-50:])/50}')
            

In [4]:
print(f"converting train_x...")
train_x = np.array(train_x_arr)
print(f"train_x shape: {train_x.shape}")

print(f"converting train_y...")
train_y = np.array(train_y_arr)
print(f"train_y shape: {train_y.shape}")


converting train_x...
train_x shape: (667, 2952, 60, 1)
converting train_y...
train_y shape: (667, 2952, 1)


In [ ]:
print(f"converting test_x...")
test_x = np.array(test_x_arr)
print(f"test_x shape: {test_x.shape}")

print("converting test_y...")
test_y = np.array(test_y_arr)
print(f"test_y shape: {test_y.shape}")
